# 商务部
- 部门职能：负责增配公司车辆资源（关注总车辆数）

- **GMV = 实收租金+异地还车费+非网点还车费+车损免赔总额+油费**

# 销售部
- **部门职能：负责拉新用户**

# 数据指标分类
## 关键指标
- 根据流程关键节点或者重要行为动作，反应业务特征和问题的本质。
- **样例：PV、UV**

## 衍生指标
- 对关键指标的重构或计算。辅助关键指标对问题的阐述，事件的刻画。
- **样例：PV/UV = 用户访问次数**

## 自建指标
- 如果不能很好诠释事件，可以根据情况自建指标。
- **样例：通过数学模型构建数据指标。**

# 运营部之网点运营
- **部门职能：负责网点车位以及网点车辆停放的管理**

## 引入数据流并加工处理

In [2]:
#导入工具包
import sys
sys.path.append("/Users/jiyanhui/Documents/Python/py")
from mysql import *
import numpy as np
import pandas as pd

#连接MySQL数据库提取数据
data = mysql("select * from dot")
data1 = mysql("select * from kkb.order")
order = mysql("select 开始网点ID as dot_id,count(用户ID) as user_num,sum(实收金额) as money from kkb.order group by 开始网点ID")

#数据加工处理
data1['品牌'] = ['上海大众' if i == '2' else i for i in data1.品牌.tolist()]
data1['订单状态'] = ['已完成' if i == '4' else i for i in data1.订单状态.tolist()]
order = order.loc[order['dot_id']>0]
data['租金'] = data['租金'].fillna(value=0)
data['车位数'] = [4 if i == 0 else i for i in data.车位数.tolist()]
data['租金'] = [400 if i == 0 else int(i) for i in data.租金.tolist()]
def function1(x):
    if x == 0:
        return "免费"
    elif x == 1:
        return "临停收费"
    elif x == 2:
        return "已租"
    elif x == 3:
        return "固定收费"
    else:
        return 'NULL'
data['是否收费'] = [function1(i) for i in data.是否收费.tolist()]
data['是否启用'] = ['启用' if i==1 else '不启用' for i in data.是否启用.tolist()]
data = data.loc[data['网点类型']==2]
new_order = pd.merge(order,data,left_on = "dot_id",right_on = '网点ID',how = "left")
new_order = pd.DataFrame(new_order)
N = new_order.iloc[:,0].size
new_order['money'] = new_order.money.fillna(value = 55)
new_order['租金'] = new_order.租金.fillna(value = 10)
new_order['new_judge'] = (new_order.money - new_order.租金)/new_order.租金

# 构建数学模型
## 一、网点的用户量和车位数之间的关系（客流量）
- **1、若用户数大于车位数*15，则该网点是高频；**
- **2、反之，则该网点为低频；**
- **3、关于15这个数值是怎么来的，在此不做详细讲解。**

## 二、网点的租金和收益之间的关系（投产比）
- **1、若网点收益大于租金的2倍，则该网点高收益；**
- **2、若网点收益大于租金金额，但是小于等于租金金额2倍，则该网点有收益；**
- **3、若网点没有收益，也就是收益金额为零，则该网点亏损。**

## 三、网点等级
- **1、若投产比是高收益，客流量是高频，则该网点为一等；**
- **2、若投产比是高收益，客流量是低频，则该网点为二等；**
- **3、若投产比是有收益，客流量是高频，则该网点为三等；**
- **4、若投产比是有收益，客流量是低频，则该网点为四等；**
- **5、若投产比是亏损，客流量是高频，则该网点为五等；**
- **6、若投产比是亏损，客流量是低频，则该网点为六等；**

In [3]:
P = []
for i in range(0,N):
    if new_order.user_num[i]>=new_order.车位数[i]:
        P.append('高频')
    else:
        P.append('低频')
new_order['客流量'] = P

PP = []
for i in range(0,N):
    if abs(new_order.new_judge[i])>1.2:
        PP.append('高收益')
    elif abs(new_order.new_judge[i])>1 and abs(new_order.new_judge[i])<=1.2:
        PP.append('有收益')
    else:
        PP.append('亏损')
new_order['投产比等级'] = PP

PPP = []
for i in np.arange(0,N):
    if new_order.投产比等级[i]=='高收益' and new_order.客流量[i] == '高频':
        PPP.append('a.一等')
    elif new_order.投产比等级[i]=='高收益' and new_order.客流量[i] == '低频':
        PPP.append('b.二等')
    elif new_order.投产比等级[i]=='有收益' and new_order.客流量[i] == '高频':
        PPP.append('c.三等')
    elif new_order.投产比等级[i]=='有收益' and new_order.客流量[i] == '低频':
        PPP.append('d.四等')
    elif new_order.投产比等级[i]=='亏损' and new_order.客流量[i] == '高频':
        PPP.append('e.五等')
    elif new_order.投产比等级[i]=='亏损' and new_order.客流量[i] == '低频':
        PPP.append('f.六等')
    else:
        PPP.append('NULL')
new_order['网点等级'] = PPP
pd.pivot_table(new_order,index=[u'网点等级'],values=[u'dot_id'],aggfunc=[len],fill_value=0)


,len
,dot_id
网点等级,
a.一等,442
b.二等,2295
c.三等,100
d.四等,20
e.五等,567
f.六等,183


## 用户等级划分和生命周期模型
- **根据用户最近一次消费时间R、一段时间的消费频次F、一段时间的消费金额M，三个变量赋予不同的权重值得出综合评分**
- **用户生命周期根据消费情况而定**

In [9]:
#导入工具包
import time
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import sys
sys.path.append("/Users/jiyanhui/Documents/Python/py")
from key import *
from mysql import *
#读取数据
data = mysql('''SELECT 用户ID,COUNT(订单ID) AS F,round(sum(实收金额),2) AS M,max(DATE_FORMAT(付款时间,'%Y-%m-%d')) as R FROM kkb.order where length(用户ID) > 2 GROUP BY 用户ID;''')

#计算R值
data['R'] = pd.to_datetime(data['R'])
dd = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
data['Datediff'] = (pd.to_datetime(dd) - data['R']).dt.days

data['RR'] = [1-(i-data.R.min())/(data.R.max()-data.R.min()) for i in data.R.tolist()]
data['FF'] = [(i-data.F.min())/(data.F.max()-data.F.min()) for i in data.F.tolist()]
data['MM'] = [(i-data.M.min())/(data.M.max()-data.M.min()) for i in data.M.tolist()]

data['score'] = 100*(0.2*data['MM'] + 0.25*data['FF'] + 0.55*data['RR'])
data['score'] = [1 if int(i) == 0 else int(i) for i in data.score.tolist()]
data['score'] = [100 if int(i) == 99 else int(i) for i in data.score.tolist()]

D = []
N = data.iloc[:,0].size
Q_median = np.median(data.loc[:,'score'])
Q_25 = np.percentile(data.loc[:,'score'],25)
Q_75 = np.percentile(data.loc[:,'score'],75)
for i in range(0,N):
    if data.loc[i,'score'] >= 1 and data.loc[i,'score'] < Q_25:
        D.append('四等')
    elif data.loc[i,'score'] >= Q_25 and data.loc[i,'score'] < Q_median:
        D.append('三等')
    elif data.loc[i,'score'] >= Q_median and data.loc[i,'score'] < Q_75:
        D.append('二等')
    elif data.loc[i,'score'] >= Q_75:
        D.append('一等')
    else:
        D.append('NULL')
data['level'] = D

data['Datediff'] = [abs(i) if i < 0 else i for i in data.Datediff.tolist()]
DD = []
for i in range(0,N):
    if data.F[i]==1 and data.Datediff[i]<=15:
        DD.append('引入期')
    elif data.F[i]>1 and data.F[i]<=3 and data.Datediff[i]<=70:
        DD.append('成长期')
    elif data.F[i]>=4 and data.Datediff[i]<=70:
        DD.append('成熟期')
    elif data.Datediff[i]>=70 and data.Datediff[i]<=90:
        DD.append('休眠期')
    elif data.Datediff[i]>90 or (data.F[i]==1 and data.Datediff[i]>15):
        DD.append('流失期')
    else:
        DD.append('else')
data['生命周期'] = DD
s = pd.pivot_table(data,index=[u'level'],values=[u'用户ID'],aggfunc=[len],fill_value=0)
s['rfm_pct'] = ["%.2f%%"%(i/sum(s.values)*100) for i in s.values]
s

,len,rfm_pct
,用户ID,
level,,
一等,9189,33.53%
三等,5968,21.78%
二等,6208,22.65%
四等,6041,22.04%


In [10]:
ss = pd.pivot_table(data,index=[u'生命周期'],values=[u'用户ID'],aggfunc=[len],fill_value=0)
ss['rfm_pct'] = ["%.2f%%"%(i/sum(ss.values)*100) for i in ss.values]

In [11]:
ss

,len,rfm_pct
,用户ID,
生命周期,,
休眠期,42,0.15%
引入期,29,0.11%
成长期,15,0.05%
流失期,27320,99.69%
